In [1]:
class Node:
  def __init__(self,feature = None,threshhold= None,left = None,right=None ,value = None):
    self.feature = feature
    self.threshhold = threshhold
    self.left = left
    self.right = right
    self.value = value

  def is_leaf_node(self):
      return self.value is not None

In [2]:
import numpy as np
from collections import Counter

In [35]:
class DecisionTree:
  def __init__(self,n_depth = 10,min_sample_split = 2,n_features = None):
      self.n_depth = n_depth
      self.min_sample_split = min_sample_split
      self.n_features = n_features
      self.root = None

  def fit(self,X,y):
    self.n_features =  X.shape[1] if not self.n_features else min(self.n_features,X.shape[1])


    # Grow the Tree with recursion
    self.root = self._grow_tree(X,y)

  def _grow_tree(self,X,y,depth = 0):


     n_samples , n_feature = X.shape
     n_label = len(np.unique(y))

     # when to stop
     print(n_samples,n_label, np.unique(y),len(y))
     if depth >= self.n_depth or n_samples < self.min_sample_split or n_label== 1: # or pure
       most_common = self._most_common(y)[0][0]
       return Node(value=most_common)


     # choose random  features
     selected_features = np.random.choice(range(X.shape[1]),self.n_features)
     best_feat,best_thresh = self._get_best_split(X,y,selected_features)



     # spliting to left and right
     left_idxs, right_idxs = self._split(X[:,best_feat],best_thresh)


     print("split feature :" ,best_feat)
     print("best_thresh :" ,best_thresh)
     print("left , right:" ,len(left_idxs), len(right_idxs))
     print("left , right:" ,self._most_common(y[left_idxs]), self._most_common(y[right_idxs]))
     print("--------------------------------------------------")

     left = self._grow_tree(X[left_idxs,:],y[left_idxs],depth+1)
     right = self._grow_tree(X[right_idxs,:],y[right_idxs],depth+1)

     return Node(best_feat,best_thresh,left,right)


  def predict(self,X):
      return np.array([self._dfs(self.root,X[i,:]) for i in range(len(X))])


  def _dfs(node,xi):
    # base
    if node.is_leaf_node():
      return node.value
    if node.threshhold <= xi[node.feature]:
      self.dfs(node.left,xi)
    else:
      self.dfs(node.right,xi)

  def _most_common(self,y):
    count = Counter(y)
    print("debug most common ",count.most_common())
    return count.most_common()

  def _get_best_split(self,X,y,X_columns):
    # calculate info gain for every threshhold and feature
    gain = -1
    best_feature = None
    best_thresh = None

    for X_column in X_columns:
      column = X[:,X_column]
      threshholds = np.unique(column) # get unique value to calculate threshhold
      for thresh in threshholds:

        # get the info gain
        info_gain = self._get_info_gain(y,column,thresh)
        if info_gain > gain:
          gain = info_gain
          best_feature = X_column
          best_thresh = thresh
    return best_feature,best_thresh

  def _split(self,X_column,threshhold):
      left = np.argwhere(X_column <= threshhold).flatten()
      right =  np.argwhere(X_column> threshhold).flatten()
      return left,right

  def _get_info_gain(self,y,X_column,threshhold):
    # greet parent entropy
    parent_entropy = self._get_entropy(y)

    # get children
    left , right = self._split(X_column,threshhold)



    # if it is pure
    if len(left) == 0 or len(right) == 0:
      return 0

    # entropy of children
    e_left = self._get_entropy(y[left])
    e_right = self._get_entropy(y[right])

    #childern weights
    wl = len(left) /len(y)
    wr = len(right) /len(y)



    info_gain = parent_entropy - (  (e_left*wl) + (e_right * wr))

    return info_gain


  def _get_entropy(self,y):
    # get probaplities
    ps = np.bincount(y)/len(y)
    return - np.sum([p*np.log(p) for p in ps])

In [36]:
dt = DecisionTree()
dt.fit(X_train,y_train)

426 2 [0 1] 426


<ipython-input-35-eb393d37bd68>:123: RuntimeWarning: divide by zero encountered in log
  return - np.sum([p*np.log(p) for p in ps])
<ipython-input-35-eb393d37bd68>:123: RuntimeWarning: invalid value encountered in double_scalars
  return - np.sum([p*np.log(p) for p in ps])


split feature : 22
best_thresh : 101.4
left , right: 238 188
debug most common  [(1, 231), (0, 7)]
debug most common  [(0, 151), (1, 37)]
left , right: [(1, 231), (0, 7)] [(0, 151), (1, 37)]
--------------------------------------------------
238 2 [0 1] 238
split feature : 28
best_thresh : 0.3849
left , right: 235 3
debug most common  [(1, 231), (0, 4)]
debug most common  [(0, 3)]
left , right: [(1, 231), (0, 4)] [(0, 3)]
--------------------------------------------------
235 2 [0 1] 235
split feature : 13
best_thresh : 44.96
left , right: 231 4
debug most common  [(1, 229), (0, 2)]
debug most common  [(1, 2), (0, 2)]
left , right: [(1, 229), (0, 2)] [(1, 2), (0, 2)]
--------------------------------------------------
231 2 [0 1] 231
split feature : 7
best_thresh : 0.05603
left , right: 227 4
debug most common  [(1, 226), (0, 1)]
debug most common  [(1, 3), (0, 1)]
left , right: [(1, 226), (0, 1)] [(1, 3), (0, 1)]
--------------------------------------------------
227 2 [0 1] 227
split 

IndexError: ignored

In [19]:
import numpy as np
from collections import Counter

class Node:
    def __init__(self, feature=None, threshold=None, left=None, right=None,*,value=None):
        self.feature = feature
        self.threshold = threshold
        self.left = left
        self.right = right
        self.value = value

    def is_leaf_node(self):
        return self.value is not None


class DecisionTree:
    def __init__(self, min_samples_split=2, max_depth=100, n_features=None):
        self.min_samples_split=min_samples_split
        self.max_depth=max_depth
        self.n_features=n_features
        self.root=None

    def fit(self, X, y):
        self.n_features = X.shape[1] if not self.n_features else min(X.shape[1],self.n_features)
        self.root = self._grow_tree(X, y)

    def _grow_tree(self, X, y, depth=0):
        n_samples, n_feats = X.shape
        n_labels = len(np.unique(y))

        # check the stopping criteria
        if (depth>=self.max_depth or n_labels==1 or n_samples<self.min_samples_split):
            leaf_value = self._most_common_label(y)
            return Node(value=leaf_value)

        feat_idxs = np.random.choice(n_feats, self.n_features, replace=False)

        # find the best split
        best_feature, best_thresh = self._best_split(X, y, feat_idxs)

        # create child nodes
        left_idxs, right_idxs = self._split(X[:, best_feature], best_thresh)
        left = self._grow_tree(X[left_idxs, :], y[left_idxs], depth+1)
        right = self._grow_tree(X[right_idxs, :], y[right_idxs], depth+1)
        return Node(best_feature, best_thresh, left, right)


    def _best_split(self, X, y, feat_idxs):
        best_gain = -1
        split_idx, split_threshold = None, None

        for feat_idx in feat_idxs:
            X_column = X[:, feat_idx]
            thresholds = np.unique(X_column)

            for thr in thresholds:
                # calculate the information gain
                gain = self._information_gain(y, X_column, thr)

                if gain > best_gain:
                    best_gain = gain
                    split_idx = feat_idx
                    split_threshold = thr

        return split_idx, split_threshold


    def _information_gain(self, y, X_column, threshold):
        # parent entropy
        parent_entropy = self._entropy(y)

        # create children
        left_idxs, right_idxs = self._split(X_column, threshold)

        if len(left_idxs) == 0 or len(right_idxs) == 0:
            return 0

        # calculate the weighted avg. entropy of children
        n = len(y)
        n_l, n_r = len(left_idxs), len(right_idxs)
        e_l, e_r = self._entropy(y[left_idxs]), self._entropy(y[right_idxs])
        child_entropy = (n_l/n) * e_l + (n_r/n) * e_r

        # calculate the IG
        information_gain = parent_entropy - child_entropy
        return information_gain

    def _split(self, X_column, split_thresh):
        left_idxs = np.argwhere(X_column <= split_thresh).flatten()
        right_idxs = np.argwhere(X_column > split_thresh).flatten()
        return left_idxs, right_idxs

    def _entropy(self, y):
        hist = np.bincount(y)
        ps = hist / len(y)
        return -np.sum([p * np.log(p) for p in ps if p>0])


    def _most_common_label(self, y):
        counter = Counter(y)
        value = counter.most_common(1)[0][0]
        return value

    def predict(self, X):
        return np.array([self._traverse_tree(x, self.root) for x in X])

    def _traverse_tree(self, x, node):
        if node.is_leaf_node():
            return node.value

        if x[node.feature] <= node.threshold:
            return self._traverse_tree(x, node.left)
        return self._traverse_tree(x, node.right)




In [20]:
dt = DecisionTree()
dt.fit(X_train,y_train)

In [22]:
  sum(dt.predict(X_test)==y_test)/len(y_test)

0.9440559440559441

In [5]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
bc = datasets.load_breast_cancer()
X,y = bc.data,bc.target
X_train, X_test, y_train, y_test = train_test_split(X,y,random_state=42,shuffle=True)

TypeError: ignored

NameError: ignored

In [16]:
a = np.array([1,1,0,0,1])
ps = np.bincount(a)/len(a)
- np.sum([p*np.log(p) for p in ps])

0.6730116670092565

In [20]:
idx = np.argwhere(a>0).flatten()

In [21]:
a[idx]

array([1, 1, 1])

In [4]:
np.random.choice([1,2,43,5],2, replace=False)

array([ 2, 43])